In [1]:
#Importation des bibliothèques 
import pandas as pd
import openrouteservice as ors
import numpy as np
import folium 

In [2]:
#fonction de construction de laa matrice des distabces 
def Matrice_des_distances(path):#elle prend le path du csv comme inoput
    file=pd.read_csv(path)
    f=file.to_numpy()
    client = ors.Client(key='5b3ce3597851110001cf624803bbfd9756c04be9aed554b68d5a1fd2')#clé du client de l'api openrouteservice
    d=[]
    for i in range (len(f)):#extraction des coordonnées deouis le csv 
        d+=[[f[i][2],f[i][3]]]
    x=[]
    for s in d:
        tuple(s)
        x.append(tuple(s)) #construction d'une liste qui contient des tuples contenant les coordonnées 
    myMATRIX = [x] + [[e] + [0] * (len(x)-1) for e in x[0:]] #construction d'une matrice vide , et qui à comme header et première colonne la liste x 
    lst=myMATRIX()
    for i in range (1,len(lst)):#on ajoute des zéros à la matrice pour fixier sa taille et avoir (len(nombre des locations)+1)^2
        lst[i].append(0)
    lst[0].insert(0,0)
    List=lst
    for i in range (1,len(lst[0])):#boucle qui fait appel au l'API et remplit la matrice construite par les distances calculées par le client 
        for j in range (1,len(lst)):       
            List[i][j]=(client.directions(tuple([List[i][0],List[0][j]]))).get("routes")[0].get("summary").get("distance")
    return List

In [3]:
L=Matrice_des_distances("C:\\Users\\WIAM\\OneDrive\\Documents\\ors.csv")

In [4]:
#Matrice qui cert d'enlever le header et la première colonne depuis la matrice des distances obtenue 
def generer_matrice_pure(List):
    T=List[1:]
    for i in range(len(T)):
        T[i].pop(0)
    return(T)
MP=generer_matrice_pure(L)
n=np.array(MP)
n[n==None]=0

In [9]:
# Required Libraries
import pandas as pd
import numpy as np
import random
import copy

# Function: Tour Distance
def distance_calc(X, city_tour):
    distance = 0
    for k in range(0, len(city_tour[0])-1):
        m = k + 1
        distance = distance + X[city_tour[0][k]-1, city_tour[0][m]-1]            
    return distance


# Seed : Generates a random list of visitation
def seed_function(X):
    seed = [[],float("inf")]
    sequence = random.sample(list(range(1,X.shape[0]+1)), X.shape[0])
    sequence.append(sequence[0])
    seed[0] = sequence
    seed[1] = distance_calc(X, seed)
    return seed


# Function: Stochastic 2_opt
def stochastic_2_opt(X, city_tour):
    best_route = copy.deepcopy(city_tour)      
    i, j  = random.sample(range(0, len(city_tour[0])-1), 2)
    if (i > j):
        i, j = j, i
    best_route[0][i:j+1] = list(reversed(best_route[0][i:j+1]))           
    best_route[0][-1]  = best_route[0][0]              
    best_route[1] = distance_calc(X, best_route)                     
    return best_route


# Function: Local Search

# iterations = Total number of iterations
# neighbourhood_size = Population size
# max_attempts = Max number of attempts to improve solution before changing the neighborhood

def local_search(X, city_tour, max_attempts = 50, neighbourhood_size = 5):
    count = 0
    solution = copy.deepcopy(city_tour) 
    while (count < max_attempts): 
        for i in range(0, neighbourhood_size):
            candidate = stochastic_2_opt(X, city_tour = solution)
        if candidate[1] < solution[1]:
            solution  = copy.deepcopy(candidate)
            count = 0
        else:
            count = count + 1                             
    return solution 

# Function: Variable Neighborhood Search
def variable_neighborhood_search(X, city_tour, max_attempts = 20, neighbourhood_size = 5, iterations = 50):
    count = 0
    solution = copy.deepcopy(city_tour)
    best_solution = copy.deepcopy(city_tour)
    while (count < iterations):
        for i in range(0, neighbourhood_size):
            for j in range(0, neighbourhood_size):
                solution = stochastic_2_opt(X, city_tour = best_solution)
            solution = local_search(X, city_tour = solution, max_attempts = max_attempts, neighbourhood_size = neighbourhood_size )
            if (solution[1] < best_solution[1]):
                best_solution = copy.deepcopy(solution) 
                break
        count = count + 1
        # print("Iteration = ", count, "-> Distance ", best_solution[1])
    return best_solution[0]

def vns(X):
  seed = seed_function(X)
  return (variable_neighborhood_search(X, city_tour = seed, max_attempts = 20, neighbourhood_size = 5, iterations = 500))




In [10]:
s=vns(n)

In [11]:
#ofonction de visualisation sur la map 
def visualiser_sur_map(List):
    f=pd.read_csv("C:\\Users\\WIAM\\OneDrive\\Documents\\ors.csv").to_numpy() #on relit le contenu du csv fournit 
    d=[]
    for i in range (len(f)):# on remplit une nouvelle liste avec des listes contenant chacune l'indice de la localisation ainsi que ces coordonés 
        d+=[[f[i][1],f[i][2],f[i][3]]]
        lst=[]

    for l in List :
        for i in range(len(d)):
            if l==int(d[i][0]):
                lst+=[tuple([d[i][2],d[i][1]])] #on fait traduire le chemin obtenu et qui donne en output des indices des villes , avec des tuples de coordonnées pour les donner à l'api de la map 
    m = folium.Map(location=[34.00106,-6.78028],zoom_start=5) #on initie la librairie follium sur le maroc 

    loc=lst

    folium.PolyLine(loc,
                color='blue',
                weight=2,
                opacity=0.8).add_to(m) #on donne à follium la liste du chemin en coordonnées obtenu et trace le chemin sur la map 
    return m

In [12]:
visualiser_sur_map(s)